In [ ]:
# Import curl module
import pycurl
import json
from io import BytesIO

images = ["base","unihub","unihubsingle", "quantumcomputing", 
	  "pytorch", "pytorchgpu", "tensorflow", "tensorflow-gpu", 
	  "chemistry", "genomics", "hydraulics", "conda", "cern-root",
	  "optimisation", "engineering", "unihubbackup", "web", "ldapproxy",
	  "ldapsyncservice"]

%run secrets.py

In [ ]:
def tagList(baseUrl,userPwd, projectName, repoName):
	url = f"{baseUrl}/projects/{projectName}/repositories/{repoName}/artifacts"
	buffer = BytesIO()
	c = pycurl.Curl()
	c.setopt(c.URL, url)
	c.setopt(c.CUSTOMREQUEST, 'GET')
	c.setopt(c.HTTPHEADER, ['Content-Type: application/json'])
	c.setopt(c.USERPWD, userPwd)
	c.setopt(c.WRITEDATA, buffer)
	c.setopt(c.TIMEOUT, 30)
	c.perform()
	c.close()

	buffer.seek(0)
	response = buffer.read().decode('utf-8')
	data = json.loads(response)
	result = []
	for artifact in data:
		if 'tags' in artifact and artifact['tags']!=None:
			for tag in artifact['tags']:
				result.append(tag['name'])
	return result

In [ ]:
def branchList():
	! gh api repos/UniNuvola/images/branches | ansi2txt > branches.json
	with open('branches.json', 'r') as file:
		branches = json.load(file)
	branchList = []

	for branch in branches:
		branchList.append(branch['name'])
	return branchList

In [ ]:
def tagRemove(baseUrl,userPwd, projectName, repoName, tagName):
	url = f"{baseUrl}/projects/{projectName}/repositories/{repoName}/artifacts/{tagName}/tags/{tagName}"
	c = pycurl.Curl()
	c.setopt(c.URL, url)
	c.setopt(c.CUSTOMREQUEST, 'DELETE')
	c.setopt(c.HTTPHEADER, ['Content-Type: application/json'])
	c.setopt(c.USERPWD, userPwd)
	c.setopt(c.TIMEOUT, 30)
	c.perform()
	c.close()

In [ ]:
branches=branchList()

In [ ]:
for image in images:
	print(f"Checking image: {image}")
	tags=tagList(baseUrl,userPwd, "uninuvola", image)
	for tag in tags:
		if tag not in branches:
			print(f"  Tag {tag} not found in branches, removing...")
			tagRemove(baseUrl,userPwd, "uninuvola", image, tag)
		else:
			print(f"  Tag {tag} found in branches, keeping...")